In [1]:
import chess
import chess.engine
import chess.pgn

In [2]:
def create_labels(pgn_file):
    pgn= open(pgn_file)
    fen_dict=dict()
    engine = chess.engine.SimpleEngine.popen_uci("stockfish/stockfish.exe")
    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break
        board=game.board()
        for move in game.mainline_moves():
            board.push(move)
            info=engine.analyse(board,chess.engine.Limit(depth=10))
            fen_dict[board.fen()]=info['score'].white()
    engine.quit()
    return fen_dict

In [3]:
fen_dict=create_labels("test.pgn")

In [6]:
def save_train_data(fen_dict):
    with open('trainData','w') as file:
        for fen in fen_dict:
            file.write(str(fen)+":"+str(fen_dict[fen])+"\n")

In [7]:
save_train_data(fen_dict)

In [12]:
def fen_to_matrix(fen):
    piece_values = {"P":1,"R":5,"N":3,"B":3,"K":100,"Q":9,"p":-1,"r":-5,"n":-3,"b":-3,"k":-100,"q":-9}
    matrix = [[0 for _ in range(8)] for _ in range(8)]
    records = fen.split()
    board = records[0]
    position_rows = board.split("/")
    contor = 0
    for position_row in position_rows:
        for potential_piece in position_row:
            if potential_piece.isalpha():
                matrix[int(contor/8)][contor%8] = piece_values[potential_piece] / piece_values["K"]
                contor+=1
            else:
                contor+= int(potential_piece)
    return matrix

In [13]:
m=fen_to_matrix("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")

In [14]:
for row in m:
    print(row)

[-0.05, -0.03, -0.03, -0.09, -1.0, -0.03, -0.03, -0.05]
[-0.01, -0.01, -0.01, -0.01, -0.01, -0.01, -0.01, -0.01]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
[0.05, 0.03, 0.03, 0.09, 1.0, 0.03, 0.03, 0.05]
